<h2>Customer Churn Analysis</h2>

Import necessary Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv(r'D:\My Prog\Naresh_i _Technology\Resume Project\Customer Churn Analysis\Churn_Modelling.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
for col in df.columns:
    print(df[col].value_counts())

Defining input var & target var

In [ ]:
x = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

### Encoding categorical var

One hot encoding the Geography column

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# # x['Geography']=encoder.fit_transform(x[['Geography']])
# one_hot_encoded = encoder.fit_transform(x[['Geography']])

# # Convert the sparse matrix to a dense array
# one_hot_array = one_hot_encoded.toarray()

# # Assign the dense array to the DataFrame column
# x['Geography'] = one_hot_array
# # 
# # one_hot_encoded = encoder.transform(df[['Category']]).toarray()

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct_col = ColumnTransformer(transformers = [('encoder',OneHotEncoder(),[1])],remainder = 'passthrough')
x = np.array(ct_col.fit_transform(x))

Label encoding the age column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2]= le.fit_transform(x[:,2])

One hot encoding the 'geography' column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

Splitting data into train and test set

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
xtrain[0]

In [ ]:
ytrain

Let's apply all the classification algorithm.
- Logistic Regression
- Naive Bayes
- Decision Trees
- Random Forest
- Support Vector Machines (SVM)
- k-Nearest Neighbors (k-NN)
- Gradient Boosting (e.g., XGBoost, LightGBM, CatBoost)

# Let's apply Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()
logit.fit(xtrain,ytrain)

In [ ]:
# Prediction
logit_pred = logit.predict(xtest)

In [ ]:
logit_pred

#### Let's Calculate all other necessary information

In [ ]:
# Accuracy
logit_ac = accuracy_score(ytest,logit_pred)
print('So the accuracy of the model is:',logit_ac)

# Bias
logit_bias = logit.score(xtrain,ytrain)
print('The bias of the model is:',logit_bias)
logit_variance = logit.score(xtest,ytest)
print('The variance of the model is:',logit_variance)

# Accuracy table
logit_cr = classification_report(ytest,logit_pred)
print('The classification report for logistic regression is:\n',logit_cr)
# table = logit.accuracy_score(ytest,logit_pred)

# Let's apply Naive Bayes using GridSearch

First on Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

In [ ]:
# parameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0],  # Smoothing parameter (Laplace smoothing)
    'fit_prior': [True, False]  # Whether to learn class prior probabilities
}

In [ ]:
#Applying gridsearch cv
grid_search = GridSearchCV(estimator = mnb,param_grid = param_grid,cv = 5)

In [ ]:
grid_search.fit(xtrain,ytrain)

In [ ]:
best = grid_search.best_params_
best

In [ ]:
mnb_pred = grid_search.predict(xtest)
mnb_pred

In [ ]:
mnb_ac = accuracy_score(ytest,mnb_pred)
mnb_ac # without cv ac= 0.544

In [ ]:
cr = classification_report(ytest,mnb_pred)
cr

In [ ]:
bias = grid_search.score(xtrain,ytrain)
bias # without cv 0.551875

In [ ]:
variance = grid_search.score(xtest,ytest)
variance # without cv 0.544

Second on Gaussian Naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(xtrain,ytrain)

In [ ]:
gnb_pred = gnb.predict(xtest)
gnb_pred

In [ ]:
gnb_ac = accuracy_score(ytest,gnb_pred)
gnb_ac

In [ ]:
bias = gnb.score(xtrain,ytrain)
bias

In [ ]:
variance = gnb.score(xtest,ytest)
variance

In [ ]:
cr = classification_report(ytest,gnb_pred)
cr

Let's go for Bernouli naive bayes algorithm

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(xtrain,ytrain)

In [ ]:
bnb_pred = bnb.predict(xtest)
bnb_pred

In [ ]:
bnb_ac = accuracy_score(ytest,bnb_pred)
bnb_ac

In [ ]:
bias = bnb.score(xtrain,ytrain)
bias

In [ ]:
variance = bnb.score(xtest,ytest)
variance

# Let's apply Decision Tree using GridSearch

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],  # Splitting criterion
    'splitter': ['best', 'random'],  # Strategy for choosing splits
    'max_depth': [None, 10, 20, 30, 40],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required at a leaf node
    'max_features': [None, 'sqrt', 'log2'],  # Number of features to consider for best split
}

In [ ]:
model = GridSearchCV(estimator = dtc, param_grid = param_grid,cv = 5)
model.fit(xtrain,ytrain)

In [ ]:
best_params = model.best_params_
best_params
'''
With cv the parameters are
{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': None,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'splitter': 'random'}
'''

In [ ]:
dtc_pred = model.predict(xtest)
dtc_pred

In [ ]:
dtc_ac = accuracy_score(ytest,dtc_pred)
dtc_ac # witout cv ac = 0.8445

In [ ]:
bias = model.score(xtrain,ytrain)
bias # without cv = 0.882125

In [ ]:
variance = model.score(xtest,ytest)
variance # without cv = 0.8445

# Let's apply Random Forest using GridSearchCv

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
#     'criterion': ['gini', 'entropy'],  # Splitting criterion for decision trees
#      'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
#     'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
#     'min_samples_leaf': [1, 2, 4],  # Minimum samples required to be at a leaf node
#     'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider for the best split
#     'bootstrap': [True, False],  # Whether to bootstrap samples when building trees
#     'oob_score': [True, False],  # Whether to use out-of-bag samples for scoring
    'random_state': [0]  # Seed for random number generation
}

In [ ]:
model = GridSearchCV(estimator=rfc,param_grid = param_grid, cv = 5)

In [ ]:
model.fit(xtrain,ytrain)

In [ ]:
best_params = model.best_params_
best_params

In [ ]:
# Prediction
rfc_pred = model.predict(xtest)
rfc_pred

In [ ]:
# Accuracy
rf_ac = accuracy_score(ytest,rfc_pred)
rf_ac # .8645 for 100 estimators , 

In [ ]:
bias = model.score(xtrain,ytrain)
bias # 1.0 for 100 est

In [ ]:
variance = model.score(xtest,ytest)
variance # .8645 for 100 est

# Let's Apply Support Vector Machine Algorithm

In [ ]:
from sklearn.svm import SVC
svc_classifier = SVC()

In [ ]:
# # Define the parameter grid
# param_grid = {
#     'C': [0.1],  # Regularization parameter
#     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Type of kernel
#     'degree': [2, 3, 4],  # Degree for 'poly' kernel
#     'gamma': ['scale', 'auto'],  # Kernel coefficient for 'rbf', 'poly', 'sigmoid'
#     'coef0': [0.0, 0.5, 1.0],  # Independent term in kernel function for 'poly', 'sigmoid'
#     'shrinking': [True, False],  # Use the shrinking heuristic
#     'probability': [True, False],  # Enable probability estimates
#     'tol': [1e-3, 1e-4, 1e-5],  # Tolerance for stopping criterion
#     'cache_size': [100, 200, 300],  # Size of the kernel cache (in MB)
#     'class_weight': [None, 'balanced'],  # Set class weights
#     'verbose': [True, False],  # Enable verbose output
#     'max_iter': [1000, 2000, -1],  # Hard limit on iterations within solver
#     'decision_function_shape': ['ovo', 'ovr'],  # Decision function shape for multi-class problems
#     'break_ties': [True, False],  # Whether to break ties according to confidence values
#     'random_state': [0, 42]  # Controls random number generation for probability estimates
# }


In [ ]:
# model = GridSearchCV(estimator = svc_classifier, param_grid=param_grid)

In [ ]:
svc_classifier.fit(xtrain,ytrain)

In [ ]:
# Prediction
svc_pred = svc_classifier.predict(xtest)
svc_pred

In [ ]:
# Accuracy
svc_ac = accuracy_score(ytest,svc_pred)
svc_ac

In [ ]:
bias = svc_classifier.score(xtrain,ytrain)
bias

In [ ]:
variance = svc_classifier.score(xtest,ytest)
variance

In [ ]:
cr = classification_report(ytest,svc_pred)
cr

# Let's Apply KNN Algorithm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [ ]:
param_grid = {
    'n_neighbors': [3, 5, 7],  # Varying number of neighbors
    'weights': ['uniform', 'distance'],  # Types of weights
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Different algorithms
    'leaf_size': [20, 30, 40],  # Varying leaf size
    'p': [1, 2],  # Power parameter for Minkowski metric
    'metric': ['euclidean', 'manhattan'],  # Distance metric options
}

In [ ]:
model = GridSearchCV(estimator=knn, param_grid = param_grid)

In [ ]:
model.fit(xtrain,ytrain)

In [ ]:
best_params = model.best_params_
best_params

In [ ]:
# Prediction
knn_pred = model.predict(xtest)
knn_pred

In [ ]:
# Accuracy
knn_ac = accuracy_score(ytest,knn_pred)
knn_ac

In [ ]:
# Bias
bias = model.score(xtrain,ytrain)
bias

In [ ]:
variance = model.score(xtest,ytest)
variance

In [ ]:
cr = classification_report(ytest,knn_pred)
cr

# Let's apply Gradient boosting algorithm

- XGBoost

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [ ]:
# Fit the model on the training data
xgb.fit(xtrain, ytrain)

# Make predictions on the test data
y_pred = xgb.predict(xtest)

In [ ]:
# Evaluate the model
xgb_ac = accuracy_score(ytest, y_pred)
xgb_ac

In [ ]:
print("Classification Report:")
xgb_cr = classification_report(ytest, y_pred)

In [ ]:
bias = xgb.score(xtrain,ytrain)
bias

In [ ]:
variance = xgb.score(xtest,ytest)
variance

In [ ]:
# # Define the parameter grid for XGBoost classifier
# param_grid = {
#     'n_estimators': [100, 200],  # Number of boosting rounds
#     'max_depth': [3, 4, 5],  # Maximum depth of a tree
#     'learning_rate': [0.1, 0.01, 0.001],  # Learning rate
#     'subsample': [0.8, 0.9, 1.0],  # Subsample ratio of the training instances
#     'colsample_bytree': [0.8, 0.9, 1.0],  # Subsample ratio of columns when constructing each tree
#     'gamma': [0, 0.1, 0.2]  # Mihttp://localhost:8888/notebooks/My%20Prog/Naresh_i%20_Technology/Resume%20Project/Customer%20Churn%20Analysis/Untitled.ipynb#nimum loss reduction required to make a further partition
# }
# '''

In [ ]:
# model = GridSearchCV(estimator=xgb, param_grid=param_grid, cv = 5)

# Let's apply lightgbm 

In [ ]:
import lightgbm as lgb
lgb_classifier = lgb.LGBMClassifier()

In [ ]:
param_grid = {
    "boosting_type": ["gbdt", "rf", "dart"],
    "num_leaves": [20, 31, 40],
    "max_depth": [5, 6, 7],
    "learning_rate": [0.05, 0.1],
    # Add more hyperparameters and their values
}

In [ ]:
model = GridSearchCV(estimator = lgb_classifier, param_grid= param_grid,cv = 5)

In [ ]:
model.fit(xtrain,ytrain)

In [ ]:
best_params = model.best_params_
best_params

In [ ]:
# Prediction
lgb_pred = model.predict(xtest)
lgb_pred

In [ ]:
# Accuracy
lgb_ac = accuracy_score(ytest,lgb_pred)
lgb_ac

In [ ]:
bias = model.score(xtrain,ytrain)
bias

In [ ]:
variance = model.score(xtest,ytest)
variance

In [ ]:
cr = classification_report(ytest,lgb_pred)
cr

# Let's apply catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cat_classifier = CatBoostClassifier()

In [ ]:
cat_classifier.fit(xtrain,ytrain)

In [ ]:
# catboost_param_grid = {
#     'iterations': [100, 200, 300],
#     'learning_rate': [0.1, 0.05, 0.01],
#     'depth': [4, 6, 8],
#     'l2_leaf_reg': [3.0, 5.0, 7.0],
#     'model_size_reg': [None, 0.1, 0.5],
#     'rsm': [0.7, 0.8, 1.0],
#     'loss_function': ['Logloss', 'CrossEntropy'],
#     'border_count': [254, 128],
#     # ... (additional parameters and their respective values)
#     'used_ram_limit': ['1.2gb', '2.4gb'],
#     'eval_fraction': [0.1, 0.2, 0.3],
#     'bagging_temperature': [0.5, 1.0, 1.5],
#     'bootstrap_type': ['Bayesian', 'Bernoulli', 'Poisson'],
#     # ... (more parameters and their respective values)
# }


In [ ]:
# model = GridSearchCV(estimator = cat_classifier,param_grid = catboost_param_grid,cv = 5)

In [ ]:
# Prediction
cat_pred = cat_classifier.predict(xtest)
cat_pred

In [ ]:
# Accuracy
cat_ac = accuracy_score(ytest,cat_pred)
cat_ac

In [ ]:
bias = cat_classifier.score(xtrain,ytrain)
bias

In [ ]:
variance = cat_classifier.score(xtest,ytest)
variance

In [ ]:
cr = classification_report(ytest,cat_pred)
cr

# Time to find the best model for our dataset

In [ ]:
algo = ['Logistic Regression','Bernouli Naive Bayes','Gaussian Naive Bayes','MultiNomial Naive Bayes','Decision Trees','Random Forest',
        'Support Vector Machines (SVM)','k-Nearest Neighbors (k-NN)','XGBoost', 'LightGBM', 'CatBoost']

In [ ]:
accuracy = [logit_ac,bnb_ac,gnb_ac,mnb_ac,dtc_ac,rf_ac,svc_ac,knn_ac,xgb_ac,lgb_ac,cat_ac]

In [ ]:
print(len(algo))
print(len(accuracy))

In [ ]:
for model, acc in zip(algo, accuracy):
    print(f"{model} model: Accuracy - {acc}")

In [ ]:
# Alternate of showing
# algo = ['Logistic Regression', 'Bernoulli Naive Bayes', 'Gaussian Naive Bayes', 'MultiNomial Naive Bayes', 'Decision Trees', 'Random Forest',
#         'Support Vector Machines (SVM)', 'k-Nearest Neighbors (k-NN)', 'XGBoost', 'LightGBM', 'CatBoost']
# accuracy = [logit_ac, bnb_ac, gnb_ac, mnb_ac, dtc_ac, rf_ac, svc_ac, knn_ac, xgb_ac, lgb_ac, cat_ac]

# for i in range(len(algo)):
#     print(f"{algo[i]}: Accuracy - {accuracy[i]}")